## Chapter 4 (Dynamic Programming)

In [2]:
import numpy as np
import pandas as pd

In [849]:
class Grid:
    def __init__(self,n,terminal_states,discount_factor=1,r=-1,actions=('left','right','top','down'),delta=0.0005):
        """
        This class creates n x n grid
        Four actions allowed Left, Right, Top and Bottom at any state
        This uses a random policy, equi-probable
        Any action results in a reward of -1
        if the actions takes out the grid, then it remains in the state
        You can define the terminal states in the grid
        """
        self.n = n
        self.terminal_states = terminal_states # tupples of row and column
        self.df = discount_factor
        # Initialize the values
        self.row = 0
        self.col = 0
        self.delta = delta
        self.VC = np.zeros((n,n))
        self.VN = np.zeros((n,n))
        self.V = [self.VC]
        self.Q = [self.VC]
        self.iteration = 0
        self.reward = {}
        self.actions= actions
        self.states = []
        self.policies = []
        self.converge = False
        self.diff = None
        self.found_optimal_policy = False
        
        self.initialize_states()
        self.create_random_policy()
        self.create_reward_function()
        
    def set_state(self,state):
        self.row = state[0]
        self.col = state[1]
        
    def initialize_states(self):
        for i in range(self.n):
            for j in range(self.n):
                self.states.append((i,j))
        
    def create_random_policy(self):
        random_policy = {}
        for state in self.states:
            if state not in self.terminal_states:
                random_policy[state] = {}
                for action in self.actions:
                    random_policy[state][action] = 1.0/len(self.actions)
        self.policies.append(random_policy)
    def create_reward_function(self):
        for state in self.states:
            self.set_state(state)
            for action in self.actions:
                target_state = self.get_sp(action)
                self.reward[state,action,target_state] = -1
    def get_sp(self,action):
        """
        This function returns the sp ( s-prime) based on current position and action
        """
        i_s = self.row
        j_s = self.col
        i_sp = i_s
        j_sp = j_s
        if action == "left":
            j_sp = j_s - 1
        if action == "right":
            j_sp = j_s + 1
        if action == "top":
            i_sp = i_s - 1
        if action=="down":
            i_sp = i_s + 1
        if i_sp < 0 or i_sp > self.n - 1:
            i_sp = i_s
        if j_sp < 0 or j_sp > self.n - 1:
            j_sp = j_s
        return i_sp,j_sp

    def calculate_state_value(self):
        """
        This function calcuates the state value and update the same
        """
        i_s = self.row
        j_s = self.col
        
        #latest optimal policy
        policy = self.policies[-1]
        
        if (i_s,j_s) in self.terminal_states:
            self.VN[i_s,j_s] = 0
        else:
            new_value = 0
            for action,p in policy[(i_s,j_s)].items():
                i_sp, j_sp = self.get_sp(action)
                r = self.reward.get(((i_s,j_s),action,(i_sp,j_sp)),0)
                new_value += p*(r+self.df*self.VC[i_sp,j_sp])
            self.VN[i_s,j_s] = new_value
    
    def update_state_values(self):
        for state in self.states:
            if not state in self.terminal_states:
                self.set_state(state)
                self.calculate_state_value()
        self.iteration += 1
        self.VC = np.array(self.VN)
        self.V.append(self.VC)      

    def check_convergence(self):
        n = len(self.V)
        if n > 2:
            diff = np.abs((self.V[n-1]-self.V[n-2]).flatten())
            converge = (diff <= self.delta).all()
            self.converge = converge
            self.diff = diff
    
    def evaluate_policy(self,delta=None,k=None):
        if delta:
            self.delta = delta
        if k is None:
            while not self.converge:
                self.update_state_values()
                self.check_convergence()
            print(f"Converged at {self.iteration}")
            print(self.diff)
        else:
            for x in range(k):
                self.update_state_values()
                self.check_convergence()
            print(self.converge)
            print(self.diff)
            
    def improve_policy(self):
        new_policy = {}
        policy = self.policies[-1]
        q_values = np.array(self.Q[-1])
        for state in self.states:
            if not state in self.terminal_states:
                new_policy[state] = {}
                actions = []
                q = []
                for action,p in policy[state].items():
                    actions.append(action)
                    self.set_state(state)
                    target_state = grid.get_sp(action)
                    prob = policy.get(state,{}).get(action,0)
                    reward = self.reward.get((state,action,target_state),0)
                    qa = prob*(reward+grid.df*grid.VC[target_state])
                    q.append(qa)
                q_max = np.max(q)
                b = [True if np.round(q_val,4)==np.round(q_max,4) else False for q_val in q]
                optimal_actions = np.array(actions)[b]
                q_values[state] = q_max
                for o_action in optimal_actions:
                    new_policy[state][o_action] = 1.0/len(optimal_actions)
        self.Q.append(q_values)
        self.policies.append(new_policy)
        self.found_optimal_policy =  self.policies[-2] == self.policies[-1]
        
    def find_optimal_policy(self):
        while not self.found_optimal_policy:
            self.evaluate_policy()
            self.improve_policy()
        print(self.policies[-1])

In [864]:
grid = Grid(4,terminal_states=[(3,3)],delta=0.0005)
#grid.iterate(2)

In [865]:
grid.find_optimal_policy()

Converged at 391
[ 0.00049027  0.00047085  0.0004408   0.00041673  0.00047085  0.00044417
  0.00039988  0.00035964  0.0004408   0.00039988  0.00032325  0.0002338
  0.00041673  0.00035964  0.0002338   0.        ]
Converged at 391
[ 0.00049027  0.00047085  0.0004408   0.00041673  0.00047085  0.00044417
  0.00039988  0.00035964  0.0004408   0.00039988  0.00032325  0.0002338
  0.00041673  0.00035964  0.0002338   0.        ]
{(0, 0): {'right': 0.5, 'down': 0.5}, (0, 1): {'right': 1.0}, (0, 2): {'down': 1.0}, (0, 3): {'down': 1.0}, (1, 0): {'down': 1.0}, (1, 1): {'right': 0.5, 'down': 0.5}, (1, 2): {'down': 1.0}, (1, 3): {'down': 1.0}, (2, 0): {'right': 1.0}, (2, 1): {'right': 1.0}, (2, 2): {'right': 0.5, 'down': 0.5}, (2, 3): {'down': 1.0}, (3, 0): {'right': 1.0}, (3, 1): {'right': 1.0}, (3, 2): {'right': 1.0}}


In [866]:
for state,action in grid.policies[-1].items():
    print(state,action)

(0, 0) {'right': 0.5, 'down': 0.5}
(0, 1) {'right': 1.0}
(0, 2) {'down': 1.0}
(0, 3) {'down': 1.0}
(1, 0) {'down': 1.0}
(1, 1) {'right': 0.5, 'down': 0.5}
(1, 2) {'down': 1.0}
(1, 3) {'down': 1.0}
(2, 0) {'right': 1.0}
(2, 1) {'right': 1.0}
(2, 2) {'right': 0.5, 'down': 0.5}
(2, 3) {'down': 1.0}
(3, 0) {'right': 1.0}
(3, 1) {'right': 1.0}
(3, 2) {'right': 1.0}


### Equations

$ V(s) = \sum \pi(s)\sum p(s',r|s,a) (r + \gamma V(s')$    $\forall a \in  A(s), s,s' \in S$

$r(s,a,s')$

$p(s,a,s')$

$q(s,a) = \sum p(s',r|s,a) (r + \gamma V(s')$    $\forall a \in  A(s), s,s' \in S$